# What is active reconnaissance?

Active reconnaissance is a process in which an individual, such as a security professional or hacker, actively engages with a target system to gather information about its vulnerabilities, configuration, and defenses. Unlike passive reconnaissance, where the attacker gathers information without directly interacting with the target system (e.g., through public sources or network sniffing without sending data to the system), active reconnaissance involves direct interaction.

## Examples: 
- port scanning
- banner grabbing
- scanning for vulnerabilities 

# Scanning ports with Scapy 

Scapy is a powerful Python library used for packet manipulation and network analysis. It allows users to create, modify, send, and intercept network packets in a programmable way, enabling extensive control over network protocols and custom packet creation. This makes Scapy a valuable tool for networking professionals, security researchers, and developers working in the fields of network testing, security audits, and forensic analysis.

In [ ]:
from scapy.all import * # this line imports all contents from the scapy library
import ipaddress # this line imports the ipaddress module used to work with IPv4 and IPv6 addresses.

host = input("Enter IP Address: ") # this line asks an IP address for scanning as input from user, which is saved in the variable host. 
ports = [ 25,80,53,443,445,8080,443] # this line represents a list with all the TCP ports used in the current SYN scanning

def synscan(host): # this line defines the synscan function, that takes host as an argument.
    ans,unans = sr(IP(dst=host)/TCP(sport=3333,dport=ports, flags="S") ,timeout=2, verbose=1) # this line calls the sr() function from scapy that sends and receives packets.This line sends SYN packets due tu "S" flag from soure port 3333, to all ports from the ports lists, from the given IP address which is the host.The scripts waits 2 seconds before sending the next request and the verbose level is 0.
    print("Open ports at %s:"%host)  # this line prints the open ports found on the target (host).
    for (s,r) in ans: # it iterates the ans tupples, where s is the send packet and r is the received packet
        if s[TCP].dport == r[TCP].sport and r[TCP].flags=="SA": # this line checks if the destination port of the send packet matches the port of the recieving packet. If the received packet has the SA ( SYN-ACK) flag set, that indicates an open port on the targeted host.
            print(s[TCP].dport) # prints the open ports founr on host.

def DNSScan(host): # it defines the DNSScan definition that takes host as an argument. 
    ans, unans = sr( IP(dst=host) / UDP(dport=53)/ DNS(rd=1, qd=DNSQR(qname="google.com")) ,timeout=2, verbose=1) # this line calls the sn() function and it sends a DNS query packet on port 53 using UDP.The domain used for query is: "google.com"
    if ans and ans[UDP]: # this line checks if there were any answers and responses that are from UDP
        print("DNS Server at %s"%host) # this line prints if the targeted host is a DNS server if the conditions met in the current def are met.

try: # initiate a try block to attempt the following code, which is sensitive to exceptions
    ipaddress.ip_address(host) # this line uses the ip_address() function from the ipaddress module, to check the given IP address from user input.If the address is not valid, this line raises an "ValueError" error
except: # initiate an except block that will catch any exceptions found in the try block.
    print("Invalid address") # if the given IP address is not valid, this information will be printed out on the screen. 
    exit(-1) # this line terminates the script with error(-1) if an error occured during the execution
synscan(host) # this line calls the synscan function defined in the first part of the script and starts a SYN scan on the given host. 
DNSScan(host) # this line calls the DNSScan function defined in the second part of the script and starts a DNSScan.



# DNS Query 

The provided Python script is designed to perform DNS queries and reverse DNS lookups using the ```dns``` library and Python's socket module. It scans for DNS records by trying different subdomains from a list stored in a file. Here's a detailed line-by-line explanation of your script:

In [ ]:
import dns # this line imports the dns module from the dnspython package
import dns.resolver # this line imoprts the dns.resolver module from the dnspython package 
import socket # this line imports the socket module

subs = "dns-scan.dns"  # this line defines the subs variable which represents the file that will be scanned for DNS 
dictionary = []  # this line represents an empty list that will store the DNS entries found during the scan.

with open(subs, "r") as f: 
    dictionary = f.read().splitlines() # these two lines are opening the sububs file in read mode. It reads all content to the dictionary by removing new line characters.This will populate the dictionary list with the subdomains that will be used later.

res = dns.resolver.Resolver() 
res.nameservers = ["8.8.8.8"]
res.port = 53 # these three lines are setting up a DNS resolver instance, in which res is the name of the instance of the dns.resolver.Resolver(). The name server is set to Google's DNS and the port used is 53, which is the standard DNS port. 

domains = {} # domains is an empty dictionary in which will be stored the DNS lookups and reverse lookups results.

def reverse_dns(ip): # this defines the reverse_dns function, that takes an ip as parameter.
    try:
        result = socket.gethostbyaddr(ip) # this line tries to resolve an IP address back to its host by using gethostbyaddr(ip)
        return [result[0]] + result[1] # if the above operation is succesfull, it returns the primary hostname and its aliases.
    except socket.herror: # this line says that if a socket.herror occur, for example an IP could not be resolved back to its host, then return an empty list.
        return [] 

def dns_request(domain): # this line defines the dns_request function, which takes domain as parameter.
    ips = [] # this line represents an empty list called ips which will store all the resolved IPs from the given domains.
    try: # this line starts a try block that will handle any exceptions during the DNS scan. 
        answers = res.resolve(domain) # this line uses the res object, which was configured earlier to use the Google's DNS, to resolve a given domain. All the results will be stored in the answer variable.
        for rdata in answers: # this line iterates over each record in answers. The rdata is a DNS resource and represents an IP (res.address) for the domain.
            ips.append(rdata.address) # this line appends the results of the above operaton to the ips list
            if rdata.address not in domains: # this line checks if the resolved IP is not already a key in the domains dictionary. 
                domains[rdata.address] = [] # If it isn't, it creates a new entry in the dictionary, with the IP address as its key and an empty list as its value. 
            domains[rdata.address].append(domain)# this line adds the current domain to the list assosiated with the resolved IP in domains dictionary.
            reverse_domains = reverse_dns(rdata.address) # this line calls the reverse_dns function on the resolved IP address. This function attempts to find the hostname associated with the IP. It returns a list of domain names associated with the IP.
            for rd in reverse_domains: # this line iterates through each domain returned by the reverse_dns function.
                if rd not in domains: # this line checks if the reverse-resolved domain, named rd is not already a key in the domains dictionary. 
                    domains[rd] = [rdata.address] #  If not, initializes a new entry in the dictionary with rd as the key and a list containing the IP address as its value.
                elif rdata.address not in domains[rd]: 
                    domains[rd].append(rdata.address)# these two lines check if the domain name rd is already in the domains dictionary but the current IP address is not in the list associated with rd, appends the IP address to this list. This ensures all related IPs are stored under their respective domain names.
    except (dns.resolver.NXDOMAIN, dns.resolver.Timeout, dns.exception.DNSException) as e: # this line egins an except block that catches specific DNS-related exceptions. 
        print(f"DNS query failed for {domain}: {e}") # If an exception is caught, this line prints an error message specifying the domain for which the DNS query failed and the exception message.
    return ips # Returns the list of IP addresses collected during the DNS queries. This list contains all the IPs resolved from the original domain query and is useful for further analysis or operations outside this function.

def host_search(domain, dictionary, nums): # this line defines the host_search function that takes 3 parameters: domain, dictionary, nums. 
    for word in dictionary: # this line iterates through each word in dictionary, appending it to the main domain, and performs a DNS request.
        subdomain = word + "." + domain
        dns_request(subdomain)
        if nums:
            for i in range(10):  # 0 to 9 , If nums is true, also appends numbers 0-9 to each subdomain and performs DNS requests.
                num_subdomain = f"{word}{i}.{domain}"
                dns_request(num_subdomain)


domain = "google.com"
host_search(domain, dictionary, True) # these two lines sets the base domain to "google.com" and calls host_search to start the DNS scanning with numerical suffixes enabled.

for domain, ips in domains.items():
    print(f"{domain}: {ips}")
